## 산술평균 기하평균

In [3]:
# 삼성전자 데이터
import FinanceDataReader as fdr
df = fdr.DataReader("005930")
df.head()

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2013-04-09,30540,30599,30200,30560,184880,NaN
2013-04-10,30800,30800,30260,30420,235594,-0.004581
2013-04-11,30420,30800,30359,30800,225830,0.012492
2013-04-12,30800,31000,30260,30340,214881,-0.014935
2013-04-15,30060,30699,30060,30460,215346,0.003955


In [5]:
# 연도마다 수익률 계산
def return_year(x):
    A = x.iloc[0]
    B = x.iloc[-1]
    return (B-A) / A

year_return = df.groupby(df.index.year)["Close"].apply(return_year)
year_return
# 2013년에 10% 떨어지고, 2014년에 1% 오르고, ...

Date
2013   -0.102094
2014    0.013751
2015   -0.052632
2016    0.495436
2017    0.411634
2018   -0.241474
2019    0.440000
2020    0.467391
2021   -0.056627
2022   -0.296438
2023    0.414414
2024   -0.331658
2025    0.131086
Name: Close, dtype: float64

In [6]:
# 산술평균
mean_return = year_return.mean()
mean_return

0.09944543396911974

In [8]:
start_price = df.iloc[0]["Close"]
end_price = df.iloc[-1]["Close"]
start_date = df.index[0]
end_date = df.index[-1]

total_years = (end_date - start_date).days / 365.25          # 4년에 1번있는 윤년 때문
total_years

12.216290212183436

In [9]:
end_price   # 삼성전자 최종 가격

60400.0

In [11]:
## 산술평균으로 수익률을 계산해서는 안된다
start_price * (1+mean_return) ** (total_years)

97306.7513552329

In [14]:
# 기하평균
start_price * (1 + cagr) ** total_years

# => 기하평균 계산 결과, 실제 삼성전자의 최종 가격과 똑같이 나옴

60399.99999999996

In [12]:
# Compound Annual Growth Rate  //  연평균 수익률 
cagr = (end_price / start_price) ** (1 / total_years) - 1
cagr

0.05735399992889989

## 종목별 연평균수익률

In [19]:
kospi = fdr.StockListing("KOSPI")  
kospi

,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
0,005930,KR7005930003,삼성전자,KOSPI,,60500,1,300,0.50,60100,60900,60000,6026551,364341894900,358138094281000,5919637922,STK
1,000660,KR7000660001,SK하이닉스,KOSPI,,287000,2,-6000,-2.05,288000,294500,285500,2106859,609320456250,208936678755000,728002365,STK
2,207940,KR7207940008,삼성바이오로직스,KOSPI,,998000,2,-1000,-0.10,1004000,1005000,995000,14549,14523173500,71031652000000,71174000,STK
3,373220,KR7373220003,LG에너지솔루션,KOSPI,,290000,2,-7000,-2.36,297500,297500,289000,76979,22526523250,67860000000000,234000000,STK
4,105560,KR7105560007,KB금융,KOSPI,,112100,1,2200,2.00,110000,112300,109700,309367,34356144150,42761901746300,381462103,STK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
957,004415,KR7004411005,서울식품우,KOSPI,,1246,1,5,0.40,1239,1246,1239,2082,2582349,2771727000,2224500,STK
958,014915,KR7014911002,성문전자우,KOSPI,,4605,2,-65,-1.39,4605,4605,4605,406,1869630,2763000000,600000,STK
959,001527,KR7001522002,동양2우B,KOSPI,,8500,1,80,0.95,8500,8500,8500,41,348500,2622836500,308569,STK
960,001525,KR7001521004,동양우,KOSPI,,4200,1,95,2.31,4200,4200,4200,872,3662400,2594470200,617731,STK


In [22]:
from tqdm import tqdm

total = []
for i in tqdm(range(len(kospi))):

    try:
        row = kospi.iloc[i]
        code = row['Code']
        name = row['Name']
        
        df = fdr.DataReader("NAVER:" + code)     # 네이버증권에서 해당코드에 대한 가격데이터 가져옴
        
        start_price = df.iloc[0]['Close']
        end_price = df.iloc[-1]['Close']
    
        start_date = df.index[0]
        end_date = df.index[-1]
        
        total_date = (end_date - start_date).days / 365.25
        cagr = (end_price / start_price) ** (1 / total_date) - 1
        
        total.append([code, name, cagr, total_date])
    
    except:
        continue

100%|████████████████████████████████████████████████████████████████████████████████| 962/962 [02:10<00:00,  7.39it/s]


In [24]:
import pandas as pd
result = pd.DataFrame(total, columns = ["Code", "Name", "CAGR", "Period"])
result = result.sort_values("CAGR", ascending = False)
result[result["Period"] > 12]

,Code,Name,CAGR,Period
54,003230,삼양식품,0.387629,12.21629
55,042700,한미반도체,0.386989,12.21629
26,138040,메리츠금융지주,0.319203,12.21629
457,003350,한국화장품제조,0.250022,12.21629
722,019175,신풍제약우,0.243732,12.21629
...,...,...,...,...
727,001470,삼부토건,-0.283434,12.21629
293,016380,KG스틸,-0.306455,12.21629
668,011810,STX,-0.313831,12.21629
865,090370,메타랩스,-0.325016,12.21629


### 연습문제 ~ 뒤쪽개념은 강사님자료에